In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score,f1_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import roc_auc_score
import six
import sys
sys.modules['sklearn.externals.six'] = six

2022-06-05 20:47:50.225953: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-05 20:47:50.226028: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
df = pd.read_csv("UNSW_undersampled_and_reversed.csv")
df

,Unnamed: 0,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,0,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,1,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,2,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,3,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,4,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185995,185995,0.000008,udp,dns,INT,2,0,114,0,125000.000300,...,15,42,0,0,0,30,42,0,Generic,1
185996,185996,0.000008,udp,dns,INT,2,0,114,0,125000.000300,...,23,35,0,0,0,35,35,0,Generic,1
185997,185997,0.000008,udp,dns,INT,2,0,114,0,125000.000300,...,23,35,0,0,0,35,35,0,Generic,1
185998,185998,0.000008,udp,dns,INT,2,0,114,0,125000.000300,...,23,35,0,0,0,35,35,0,Generic,1


In [4]:
from sklearn.utils import shuffle
df = shuffle(df,random_state=50)

In [5]:
from sklearn import preprocessing

In [6]:
label_encoder1 = preprocessing.LabelEncoder()
df['proto']= label_encoder1.fit_transform(df['proto'])
  
df['proto'].unique()
label_encoder2 = preprocessing.LabelEncoder()
df['attack_cat']= label_encoder2.fit_transform(df['attack_cat'])
  
df['attack_cat'].unique()
label_encoder3 = preprocessing.LabelEncoder()
df['service']= label_encoder3.fit_transform(df['service'])
  
df['service'].unique()
label_encoder4 = preprocessing.LabelEncoder()
df['state']= label_encoder4.fit_transform(df['state'])
  
df['state'].unique()

array([4, 3, 1, 6, 2, 7, 0, 5, 8, 9])

In [7]:
y=df['attack_cat']
#y1=df.iloc[:,-1]
x_k=df.drop(['label','attack_cat','Unnamed: 0'],axis=1)
#y= df['attack_cat']

In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_l = le.fit_transform(y)
y_l

array([5, 6, 6, ..., 5, 0, 5])

In [29]:
pd.unique(y)

array([5, 6, 7, 2, 4, 0, 3, 1, 8, 9])

In [9]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
x_k_1= sc.fit_transform(x_k)

x = pd.DataFrame(x_k_1)

x.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,1.333334e-07,0.901515,0.2,0.444444,0.001451,0.000000,0.000265,0.000000,0.125000,0.996078,...,0.310345,0.275862,0.355556,0.532258,0.0,0.0,0.0,0.288136,0.524590,0.0
1,1.581304e-02,0.856061,0.0,0.333333,0.175617,0.073427,0.022980,0.006727,0.000260,0.121569,...,0.034483,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.016949,0.180328,0.0
2,7.977668e-04,0.856061,0.0,0.333333,0.053701,0.023310,0.007119,0.008228,0.001609,0.121569,...,0.068966,0.000000,0.000000,0.016129,0.0,0.0,0.0,0.033898,0.245902,0.0
3,1.500000e-07,0.901515,0.2,0.444444,0.001451,0.000000,0.000265,0.000000,0.111111,0.996078,...,0.275862,0.258621,0.333333,0.532258,0.0,0.0,0.0,0.254237,0.524590,0.0
4,1.333334e-07,0.901515,0.0,0.444444,0.001451,0.000000,0.000425,0.000000,0.125000,0.996078,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.050847,0.000000,0.0


In [13]:
from sklearn.model_selection import train_test_split

In [14]:
from sklearn.model_selection import RandomizedSearchCV

In [18]:
from sklearn.ensemble import RandomForestClassifier

In [13]:
parameters_RF = {'n_estimators': (40,60,65,70,75,80,85,90,95,100,120),'min_samples_split' : (2,4,8,10,15,20,30)}
                                                                                                                                     
RF_grid  = RandomizedSearchCV( RandomForestClassifier(), param_distributions = parameters_RF, cv = 5, verbose = True,scoring='accuracy')
result_RF=RF_grid.fit(x,y)
  
RF= result_RF.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [14]:
RF_estimator_result =result_RF.best_estimator_
print(RF_estimator_result)

RandomForestClassifier(min_samples_split=30, n_estimators=75)


In [19]:
from sklearn.naive_bayes import GaussianNB

In [16]:
parameters_gb = {'var_smoothing': (1e-09,1e-08,1e-07,1e-06,1e-05,1e-04,1e-03,1e-02,1e-01,1,10,5e-09,5e-08,5e-07,5e-06,5e-05,
                                5e-04,5e-03,5e-02,5e-01,5)}
gb_grid  = RandomizedSearchCV( GaussianNB(), param_distributions = parameters_gb, cv = 5, verbose = True,scoring='accuracy')
result_gb = gb_grid.fit(x,y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [17]:
gb_estimator_result =result_gb.best_estimator_
print(gb_estimator_result)

GaussianNB(var_smoothing=0.1)


In [20]:
from sklearn.linear_model import LogisticRegression

In [19]:
parameters_lr = {'solver':('liblinear','newton-cg'),'random_state':(0,1),'C': (0.1,1,10),'tol': (0.01,0.001,0.0001,.1)}
                                                                                                                                     
lr_grid  = RandomizedSearchCV( LogisticRegression(), param_distributions = parameters_lr, cv = 5, verbose = True,scoring='accuracy')
result_lr=lr_grid.fit(x,y)
    
LG_estimator_result = result_lr.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [20]:
print(LG_estimator_result)

LogisticRegression(C=10, random_state=0, solver='newton-cg')


In [21]:
from sklearn.neighbors import KNeighborsClassifier

In [22]:
parameters_knn = {'n_neighbors': (1,2,3,4,5),'leaf_size':(15,30,50,100),'p':(1,2,3)}
knn_grid  = RandomizedSearchCV( KNeighborsClassifier(), param_distributions = parameters_knn, cv = 5, verbose = True,scoring='accuracy')
result_knn=knn_grid.fit(x,y)
knn_estimator_result = result_knn.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [23]:
print(knn_estimator_result)

KNeighborsClassifier(leaf_size=15, p=1)


In [22]:
from sklearn.tree import DecisionTreeClassifier

In [31]:
parameters_dt = {'min_samples_split' : (2,4,6,10), 'max_depth':(3,4,5,6,7,8,9,10,11,12,16,25,50,100,200)}                                                                                                                            
DT_grid  = RandomizedSearchCV( DecisionTreeClassifier(), param_distributions = parameters_dt, cv = 5, verbose = True,scoring='accuracy')
result_dt=DT_grid.fit(x,y)
    
dt_estimator_result = result_dt.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [32]:
print(dt_estimator_result)

DecisionTreeClassifier(max_depth=12, min_samples_split=4)


In [10]:
from xgboost import XGBClassifier

In [11]:
import xgboost as xgb

In [15]:
parameters_xgb = {'alpha': (0.1,0.5,1,2,3,4,5),'eta': (0.3,0.001,0.1,0.5,1,3,5),'lambda': (0.1,1,5,3,0.5),
'colsample_bytree': [0.1, 0.5, 0.8, 1], 'subsample': [0.1,0.5,1]}
xgb_grid  = RandomizedSearchCV( XGBClassifier(), param_distributions = parameters_xgb, cv = 5, verbose = True,scoring='accuracy')
result_xgb=xgb_grid.fit(x,y_l)
    
xgb_estimator_result = result_xgb.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


KeyboardInterrupt: 

In [ ]:
print(xgb_estimator_result)

In [20]:
from sklearn.ensemble import GradientBoostingClassifier

In [43]:
parameters_grb = {'n_estimators': (50,100,120,140,150),'min_samples_split' : (2,4),'subsample' : (0.1,0.5,1)}
                                                                                                                                     
grb_grid  = RandomizedSearchCV( GradientBoostingClassifier(learning_rate=10000), param_distributions = parameters_grb, cv = 5, verbose = 50,scoring='accuracy')
result=grb_grid.fit(x,y)
    
grb_estimator_result = result.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5; 1/10] START min_samples_split=2, n_estimators=140, subsample=1.........
[CV 1/5; 1/10] END min_samples_split=2, n_estimators=140, subsample=1;, score=0.116 total time= 9.9min
[CV 2/5; 1/10] START min_samples_split=2, n_estimators=140, subsample=1.........
[CV 2/5; 1/10] END min_samples_split=2, n_estimators=140, subsample=1;, score=0.828 total time= 9.9min
[CV 3/5; 1/10] START min_samples_split=2, n_estimators=140, subsample=1.........
[CV 3/5; 1/10] END min_samples_split=2, n_estimators=140, subsample=1;, score=0.095 total time=10.2min
[CV 4/5; 1/10] START min_samples_split=2, n_estimators=140, subsample=1.........
[CV 4/5; 1/10] END min_samples_split=2, n_estimators=140, subsample=1;, score=0.018 total time= 9.9min
[CV 5/5; 1/10] START min_samples_split=2, n_estimators=140, subsample=1.........
[CV 5/5; 1/10] END min_samples_split=2, n_estimators=140, subsample=1;, score=0.038 total time=10.2min
[CV 1/5; 2/10] STAR

In [44]:
print(grb_estimator_result)

GradientBoostingClassifier(learning_rate=10000, n_estimators=140, subsample=0.1)


In [16]:
from sklearn.ensemble import AdaBoostClassifier

In [17]:
parameters_adaboost = {
    'n_estimators': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 20],
    'learning_rate': [(0.97 + x / 100) for x in range(0, 8)],
    'algorithm': ['SAMME', 'SAMME.R'],
    'random_state': [0, 1, 2]
}

adaboost_rcv = RandomizedSearchCV(
    AdaBoostClassifier(),
    param_distributions=parameters_adaboost,
    verbose=50,
    cv=5,
    scoring='accuracy'
)

adaboost_result = adaboost_rcv.fit(x, y_l)

print(adaboost_result.best_estimator_)
print(adaboost_result.best_score_)
print(adaboost_result.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5; 1/10] START algorithm=SAMME.R, learning_rate=1.03, n_estimators=9, random_state=2
[CV 1/5; 1/10] END algorithm=SAMME.R, learning_rate=1.03, n_estimators=9, random_state=2;, score=0.864 total time=   4.1s
[CV 2/5; 1/10] START algorithm=SAMME.R, learning_rate=1.03, n_estimators=9, random_state=2
[CV 2/5; 1/10] END algorithm=SAMME.R, learning_rate=1.03, n_estimators=9, random_state=2;, score=0.863 total time=   3.9s
[CV 3/5; 1/10] START algorithm=SAMME.R, learning_rate=1.03, n_estimators=9, random_state=2


KeyboardInterrupt: 

In [48]:
pip install keras-tuner 

Note: you may need to restart the kernel to use updated packages.


In [18]:
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
import keras_tuner as kt

In [19]:
X_train_ann, X_test_ann, y_train_ann, y_test_ann = train_test_split(x_k_1, y_l, test_size=0.2, random_state=75)

In [23]:
y_test_ann.shape

(37200,)

In [22]:
def build_model_expo_with_all_final_layer_act_with_adagrad_optimizer(hp):
  #with tf.device('/device:GPU:0'):
    model_expo = keras.Sequential() 
    #model_expo.add(layers.Dense(30, input_dim=4, activation='relu'))
    model_expo.add(layers. Flatten (input_dim=42))
    
    model_expo.add(layers.Dense(units=hp.Int('units',
    					 min_value=25 ,
 					 max_value=150,
   					 step=1),
			   activation=hp.Choice('act_1', ['relu', 'sigmoid','softmax','softplus','softsign','tanh','selu','elu','exponential'])))

    model_expo.add(layers.Dense(10, activation=hp.Choice('act_2', ['relu', 'sigmoid','softmax','tanh','exponential'])))
    model_expo.compile(optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', 
       		      values=[.01,.05,.5,.001,.99,5.1])), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return model_expo

In [30]:
tuner = kt.RandomSearch(
    build_model_expo_with_all_final_layer_act_with_adagrad_optimizer,
    objective='val_accuracy', 
    max_trials=2500,
    executions_per_trial=1,
    #overwrite=True,
    project_name='ann_adagrad_with_exponential_default2.1')


2022-06-05 22:09:20.209190: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-05 22:09:20.210214: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lizard-Lenovo-ideapad-320-15IKB): /proc/driver/nvidia/version does not exist
2022-06-05 22:09:20.214304: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [31]:
tuner.search(X_train_ann, y_train_ann, epochs=5, validation_data=(X_test_ann, y_test_ann))
best_model = tuner.get_best_models()[0]


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
83                |?                 |units
softplus          |?                 |act_1
exponential       |?                 |act_2
0.01              |?                 |learning_rate

Epoch 1/5
2038/4650 [============>.................] - ETA: 6s - loss: 0.5450 - accuracy: 0.8247

KeyboardInterrupt: 